# RNN for semtiment analysis

**初期設定**

In [1]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [3]:
tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')

**Config Info**

In [4]:
np.random.seed(7)
# Load the dataset but only keep the top n words, zero the rest
top_words = 10000
# Trancate and pad input sequences
max_review_length = 80
(x_tr, y_tr), (x_te, y_te) = keras.datasets.imdb.load_data(num_words=top_words)
print("Pad sequences (samples x time)")
x_tr = keras.preprocessing.sequence.pad_sequences(x_tr, maxlen=max_review_length)
x_te = keras.preprocessing.sequence.pad_sequences(x_te, maxlen=max_review_length)
print('x_trian shape: ',x_tr.shape)
print('x_test shape: ',x_te.shape)

17465344/17464789 [==============================] - 0s 0us/step
Pad sequences (samples x time)
x_trian shape:  (25000, 80)
x_test shape:  (25000, 80)


In [9]:
class RNN(keras.Model):
    def __init__(self, units, num_class, num_layers):
        super(RNN, self).__init__()
        
        self.rnn = keras.layers.LSTM(units, return_sequences=True)
        self.rnn2 = keras.layers.LSTM(units)
        
        self.embedding = keras.layers.Embedding(top_words, 100, input_length=max_review_length)
        self.fc = keras.layers.Dense(1)
        
    def call(self, inputs, training=None, mask=None):
        x = self.embedding(inputs)
        x = self.rnn(x)
        x = self.rnn2(x)
        
        x = self.fc(x)
        print(x.shape)

        return x 

In [7]:
def main():
    units = 64
    num_classes = 2
    batch_size = 32
    epochs = 20
    
    model = RNN(units, num_classes, num_layers=2)
    
    model.compile(optimizer=keras.optimizers.Adam(0.001),
                  loss=keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    model.fit(x_tr, y_tr, batch_size=batch_size, epochs=epochs,
              validation_data=(x_te, y_te), verbose=1)
    
    scores = model.evaluate(x_te, y_te, batch_size, verbose=1)
    
    print('Final test loss and accuracy: ', scores)

In [10]:
if __name__ == '__main__':
    main()

Epoch 1/20
(None, 1)
(None, 1)
782/782 [==============================] - 14s 18ms/step - loss: 0.4217 - accuracy: 0.7921 - val_loss: 0.3665 - val_accuracy: 0.8286
Epoch 2/20
782/782 [==============================] - 14s 18ms/step - loss: 0.2673 - accuracy: 0.8857 - val_loss: 0.3763 - val_accuracy: 0.8424
Epoch 3/20
782/782 [==============================] - 14s 18ms/step - loss: 0.1887 - accuracy: 0.9235 - val_loss: 0.4449 - val_accuracy: 0.8334
Epoch 4/20
782/782 [==============================] - 14s 18ms/step - loss: 0.1296 - accuracy: 0.9508 - val_loss: 0.4835 - val_accuracy: 0.8279
Epoch 5/20
782/782 [==============================] - 14s 18ms/step - loss: 0.0854 - accuracy: 0.9679 - val_loss: 0.5462 - val_accuracy: 0.8268
Epoch 6/20
782/782 [==============================] - 14s 18ms/step - loss: 0.0546 - accuracy: 0.9806 - val_loss: 0.7347 - val_accuracy: 0.8199
Epoch 7/20
782/782 [==============================] - 14s 18ms/step - loss: 0.0432 - accuracy: 0.9854 - val_loss: 0.